In [1]:
from kiteconnect import KiteConnect

In [2]:
import logging
import requests

In [3]:
import pandas as pd

In [4]:
from time import time, sleep

In [5]:
from datetime import datetime, timedelta

In [6]:
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [8]:
api_key = "f5ean4aaxq3wv9n3"
client_id = "ED5107"
api_secret = "s1kbs5y654zhbpqgd2rvslqc6shx9yh8"

In [9]:
logging.basicConfig(level=logging.DEBUG)
kite = KiteConnect(api_key=api_key)
print(kite.login_url())

https://kite.trade/connect/login?api_key=f5ean4aaxq3wv9n3&v=3


In [10]:
# Get the Request Token from the redirect URL

request_token = "WbhIjx7WlQxJwC1ir3Px1qqb44bDM2du"

# Get Access Token

In [11]:
#data = kite.generate_session(request_token, api_secret=api_secret)
#access_token = data["access_token"]
access_token = 'i69P4YjMJZ50wxtnAjUfwydHfc8q0Xvi'
kite.set_access_token(access_token)
print(access_token)

i69P4YjMJZ50wxtnAjUfwydHfc8q0Xvi


In [12]:
print(kite.profile())

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /user/profile HTTP/1.1" 200 None


{'user_id': 'ED5107', 'user_type': 'individual', 'email': 'ramanathannalini@gmail.com', 'user_name': 'Nalini Ramanathan', 'user_shortname': 'Nalini', 'broker': 'ZERODHA', 'exchanges': ['BSE', 'BFO', 'CDS', 'NFO', 'MF', 'NSE'], 'products': ['CNC', 'NRML', 'MIS', 'BO', 'CO'], 'order_types': ['MARKET', 'LIMIT', 'SL', 'SL-M'], 'avatar_url': '', 'meta': {'demat_consent': 'consent'}}


# Get Instrument Tokens

In [13]:
def get_instrument_token(tradingsymbol, instrument_type, exchange):
    instruments = pd.DataFrame(kite.instruments())
    instrument_token = instruments[(instruments['tradingsymbol']==tradingsymbol) & (instruments['instrument_type']==instrument_type) & (instruments['exchange']==exchange)]['instrument_token'].to_list()[0]
    return str(instrument_token)

# Get Historical Data

In [14]:
def get_historical_data(instrument_token, from_date, to_date, interval):
    return pd.DataFrame(kite.historical_data(instrument_token=instrument_token,
                                    from_date=from_date, 
                                   to_date=to_date,
                                   interval=interval))

# Get Exponential Moving Average

In [15]:
def get_exponential_moving_avg(timeseries, span):
    """
    adjust=False specifies that we are interested in the recursive calculation mode.
    """
    timeseries[str(span)+'d_EMAvg'] = timeseries.iloc[:,0].ewm(span=span, adjust=False).mean()
    return timeseries

# Stochastic Momentum Indicator

In [16]:
def get_smi(hist_data, k_period = 10, k_smooth_period = 3, d_smooth_period = 3, d_period = 10):
    close = pd.DataFrame({'date':[datetime.strftime(d, "%Y-%m-%d %H:%M:%S") for d in list(hist_data['date'])],'close':list(hist_data['close'])})
    highest_high = pd.DataFrame(data=list(hist_data['high']), columns=['highest_high']).rolling(k_period, min_periods=1).max()
    lowest_low = pd.DataFrame(data=list(hist_data['low']), columns=['lowest_low']).rolling(k_period, min_periods=1).min()
    mean_hl = pd.DataFrame(data=(highest_high['highest_high']+lowest_low['lowest_low'])*0.5, columns=['mean_hl'])
    hh_minus_ll = pd.DataFrame(data=(highest_high['highest_high']-lowest_low['lowest_low']), columns=['hh_minus_ll'])
    close_minus_mean_hl = pd.DataFrame(data=(close['close']-mean_hl['mean_hl']), columns=['close_minus_mean_hl'])
    emavg_cmmhl = pd.DataFrame(data=list(close_minus_mean_hl['close_minus_mean_hl'].ewm(span=k_smooth_period, adjust=False).mean()), columns=['emavg_cmmhl'])
    emavg_emavg_cmmhl = pd.DataFrame(data=list(emavg_cmmhl['emavg_cmmhl'].ewm(span=k_smooth_period, adjust=False).mean()), columns=['emavg_emavg_cmmhl'])
    emavg_hhmll = pd.DataFrame(data=list(hh_minus_ll['hh_minus_ll'].ewm(span=d_smooth_period, adjust=False).mean()), columns=['emavg_hhmll'])
    emavg_emavg_hhmll = pd.DataFrame(data=list(emavg_hhmll['emavg_hhmll'].ewm(span=d_smooth_period, adjust=False).mean()), columns=['emavg_emavg_hhmll'])
    smi = pd.DataFrame(data=100*(emavg_emavg_cmmhl['emavg_emavg_cmmhl']/(emavg_emavg_hhmll['emavg_emavg_hhmll']/2)), columns=['smi'])
    signal = pd.DataFrame(data=list(smi['smi'].ewm(span=d_period, adjust=False).mean()), columns=['signal'])
    smi_df = pd.concat([close, smi, signal],axis=1)
    smi_df.set_index('date', drop=True, inplace=True)
    return smi_df

# Zerodha Testing LIVE Deployment TradeLog

In [17]:
def place_trade(trade_log, current_time, current_price, current_4davg, current_14davg, current_transaction_type):
    if len(trade_log)==0 or trade_log.loc[list(trade_log.index)[-1], 'status']=='closed':
        current_status = 'open'
    elif trade_log.loc[list(trade_log.index)[-1], 'status']=='open':
        current_status = 'closed'
    trade_log.loc[len(trade_log)] = [current_time, current_price, current_4davg, current_14davg, current_transaction_type, current_status]
    print(current_time, current_transaction_type, current_price, current_status)
    return trade_log, current_transaction_type, current_status

# Pre-Flight Checks

### WARNING: Don't forget to update the date and time in 'deply_till' variable

In [23]:
#INSTRUMENT DATA
tradingsymbol = 'BANKNIFTY20SEPFUT'
instrument_type = 'FUT'
exchange = 'NFO'

# TRADE DETAILS
qty = 1
order_type='MARKET'
product='MIS'
variety='regular'

# CHART SPECIFICATIONS
candle_size = '3minute'
k_period = 10
k_smooth_period = 3
d_smooth_period = 3
d_period = 10

# DEPLOYMENT DURATION
deploy_till = datetime(year=2020, month=9, day=3, hour=15 , minute=30 , second=0)

# first transaction time
next_transaction_time = datetime(2020, 9, 4, 13, 59, 58)
candle = 3

# THE PAYLOAD

In [19]:
instoken = get_instrument_token(tradingsymbol, instrument_type, exchange)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments HTTP/1.1" 200 863438


In [24]:
price_log = pd.DataFrame(columns=['time','price','4d_EMAvg','14d_EMAvg'])
trade_log = pd.DataFrame(columns=['time','price','4d_EMAvg','14d_EMAvg','transaction_type','status'])
current_status='closed'
while datetime.strftime(datetime.now(), "%H:%M:%S")!=datetime.strftime(deploy_till-timedelta(minutes=1), "%H:%M:%S"):
    sleep(1)
    test_end = datetime.now()
    test_start = test_end - timedelta(days=1)
    print(test_end)
    if datetime.strftime(test_end, "%H:%M:%S")==datetime.strftime(deploy_till-timedelta(minutes=2), "%H:%M:%S"):
        for _ in range(5):
            try:
                hist_data = get_historical_data(instoken, test_start, test_end, interval=candle_size)
                break
            except requests.exceptions.ReadTimeout as e:
                logging.exception("Timeout Error - while getting ltp: \n %s", e)
                logging.info("Retrying")
                continue
        emav_df = pd.DataFrame({'date':[datetime.strftime(d, "%Y-%m-%d %H:%M:%S") for d in list(hist_data['date'])],'price':list(hist_data['close'])}).set_index('date', drop=True)
        for window in [4, 14]:
            emav_df = get_exponential_moving_avg(emav_df, window)
        current_time = list(emav_df.index)[-1]
        current_price = emav_df.loc[current_time, 'price']
        current_4davg = emav_df.loc[current_time, '4d_EMAvg']
        current_14davg = emav_df.loc[current_time, '14d_EMAvg']
        price_log.loc[len(price_log)]=[current_time, current_price, current_4davg, current_14davg]
        if current_status=='open':
            if current_transaction_type=='buy':
                transaction_type='sell'
                trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_4davg=current_4davg, current_14davg=current_14davg, current_transaction_type=transaction_type)
                break
            if current_transaction_type=='sell':
                transaction_type='buy'
                trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_4davg=current_4davg, current_14davg=current_14davg, current_transaction_type=transaction_type)
                break
        else:
            break
    
    if test_end.hour==next_transaction_time.hour and test_end.minute==next_transaction_time.minute and test_end.second>=next_transaction_time.second:
        for _ in range(5):
            try:
                hist_data = get_historical_data(instoken, test_start, test_end, interval=candle_size)
                break
            except requests.exceptions.ReadTimeout as e:
                logging.exception("Timeout Error - while getting ltp: \n %s", e)
                logging.info("Retrying")
                continue
        
        emav_df = pd.DataFrame({'date':[datetime.strftime(d, "%Y-%m-%d %H:%M:%S") for d in list(hist_data['date'])],'price':list(hist_data['close'])}).set_index('date', drop=True)
        for window in [4, 14]:
            emav_df = get_exponential_moving_avg(emav_df, window)
        current_time = datetime.strftime(next_transaction_time-timedelta(minutes=candle-1, seconds=58), "%Y-%m-%d %H:%M:%S")
        current_price = emav_df.loc[current_time, 'price']
        current_4davg = emav_df.loc[current_time, '4d_EMAvg']
        current_14davg = emav_df.loc[current_time, '14d_EMAvg']
        price_log.loc[len(price_log)]=[current_time, current_price, current_4davg, current_14davg]
        
        if current_4davg>current_14davg:
            transaction_type='buy'
            if current_status=='closed':
                trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_4davg=current_4davg, current_14davg=current_14davg, current_transaction_type=transaction_type)
            elif current_status=='open' and current_transaction_type=='sell':
                trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_4davg=current_4davg, current_14davg=current_14davg, current_transaction_type=transaction_type)
                trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_4davg=current_4davg, current_14davg=current_14davg, current_transaction_type=transaction_type)
        elif current_4davg<current_14davg:
            transaction_type='sell'
            if current_status=='closed':
                trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_4davg=current_4davg, current_14davg=current_14davg, current_transaction_type=transaction_type)
            elif current_status=='open' and current_transaction_type=='buy':
                trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_4davg=current_4davg, current_14davg=current_14davg, current_transaction_type=transaction_type)
                trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_4davg=current_4davg, current_14davg=current_14davg, current_transaction_type=transaction_type)
        next_transaction_time = next_transaction_time+timedelta(minutes=candle)

2020-09-04 13:59:08.277264
2020-09-04 13:59:09.278687
2020-09-04 13:59:10.278695
2020-09-04 13:59:11.280137
2020-09-04 13:59:12.281539
2020-09-04 13:59:13.282962
2020-09-04 13:59:14.284407
2020-09-04 13:59:15.286143
2020-09-04 13:59:16.287568
2020-09-04 13:59:17.288991
2020-09-04 13:59:18.290414
2020-09-04 13:59:19.291837
2020-09-04 13:59:20.292772
2020-09-04 13:59:21.294195
2020-09-04 13:59:22.295619
2020-09-04 13:59:23.297062
2020-09-04 13:59:24.297466
2020-09-04 13:59:25.297889
2020-09-04 13:59:26.299313
2020-09-04 13:59:27.300738
2020-09-04 13:59:28.301763
2020-09-04 13:59:29.302167
2020-09-04 13:59:30.303589
2020-09-04 13:59:31.304034
2020-09-04 13:59:32.304439
2020-09-04 13:59:33.305861
2020-09-04 13:59:34.306285
2020-09-04 13:59:35.307708
2020-09-04 13:59:36.308132
2020-09-04 13:59:37.309556
2020-09-04 13:59:38.310979
2020-09-04 13:59:39.312403
2020-09-04 13:59:40.313826
2020-09-04 13:59:41.315252
2020-09-04 13:59:42.316676
2020-09-04 13:59:43.318097
2020-09-04 13:59:44.319541
2

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443


2020-09-04 13:59:58.336450


DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/14956290/3minute?instrument_token=14956290&from=2020-09-03+13%3A59%3A58&to=2020-09-04+13%3A59%3A58&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2569


2020-09-04 13:57:00 sell 23295.0 open
2020-09-04 14:00:00.143411
2020-09-04 14:00:01.144834
2020-09-04 14:00:02.146259
2020-09-04 14:00:03.147680
2020-09-04 14:00:04.149103
2020-09-04 14:00:05.150527
2020-09-04 14:00:06.151953
2020-09-04 14:00:07.153374
2020-09-04 14:00:08.154797
2020-09-04 14:00:09.156241
2020-09-04 14:00:10.157644
2020-09-04 14:00:11.159088
2020-09-04 14:00:12.159493
2020-09-04 14:00:13.160915
2020-09-04 14:00:14.162338
2020-09-04 14:00:15.162765
2020-09-04 14:00:16.163186
2020-09-04 14:00:17.164609
2020-09-04 14:00:18.166033
2020-09-04 14:00:19.167478
2020-09-04 14:00:20.167884
2020-09-04 14:00:21.169304
2020-09-04 14:00:22.170728
2020-09-04 14:00:23.172153
2020-09-04 14:00:24.173575
2020-09-04 14:00:25.175018
2020-09-04 14:00:26.175442
2020-09-04 14:00:27.175847
2020-09-04 14:00:28.177290
2020-09-04 14:00:29.177693
2020-09-04 14:00:30.179120
2020-09-04 14:00:31.180542
2020-09-04 14:00:32.181966
2020-09-04 14:00:33.183408
2020-09-04 14:00:34.183811
2020-09-04 14:00:

...

2020-09-04 15:27:00.173416
2020-09-04 15:27:01.174840
2020-09-04 15:27:02.175267
2020-09-04 15:27:03.176687
2020-09-04 15:27:04.178113
2020-09-04 15:27:05.178535
2020-09-04 15:27:06.179962
2020-09-04 15:27:07.181382
2020-09-04 15:27:08.182806
2020-09-04 15:27:09.184249
2020-09-04 15:27:10.184652
2020-09-04 15:27:11.186076
2020-09-04 15:27:12.187499
2020-09-04 15:27:13.188923
2020-09-04 15:27:14.190346
2020-09-04 15:27:15.190772
2020-09-04 15:27:16.192215
2020-09-04 15:27:17.192638
2020-09-04 15:27:18.194041
2020-09-04 15:27:19.195467
2020-09-04 15:27:20.196889
2020-09-04 15:27:21.198311
2020-09-04 15:27:22.199738
2020-09-04 15:27:23.201159
2020-09-04 15:27:24.202582
2020-09-04 15:27:25.204005
2020-09-04 15:27:26.205429
2020-09-04 15:27:27.206873
2020-09-04 15:27:28.208276
2020-09-04 15:27:29.209700
2020-09-04 15:27:30.211123
2020-09-04 15:27:31.212546
2020-09-04 15:27:32.213972
2020-09-04 15:27:33.215886
2020-09-04 15:27:34.217309
2020-09-04 15:27:35.218734
2020-09-04 15:27:36.220156
2

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443


2020-09-04 15:28:00.253318


DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/14956290/3minute?instrument_token=14956290&from=2020-09-03+15%3A28%3A00&to=2020-09-04+15%3A28%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2611


2020-09-04 15:27:00 sell 23072.45 closed


In [25]:
trade_log.to_excel('LIVE Deployment Logs\RUN1B-HD-20200904-3min_candle_prev_candle_close_4vs14_emavg.xlsx')